In [12]:
# -*- coding: utf-8 -*-
"""
Created on Wed Apr 28 09:11:04 2021

@author: aurel
"""
import torch
import dlc_practical_prologue as prologue

from torch import optim
from torch.nn import functional as F
from torch import nn
from torch.autograd import Variable


######################################################################
def compute_nb_errors(model, data_input, data_target, mini_batch_size):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        _, _, result = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(result, 1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [28]:
def compute_nb_errors_comparison(model, data_input, data_target, data_classes, mini_batch_size):
    
    nb_data_errors = 0
    nb_errors_comparison = 0
    nb_errors_digit1 = 0
    nb_errors_digit2 = 0
    
    for b in range(0, data_input.size(0), mini_batch_size):
        digit1, digit2, result = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(result, 1)
        _, predicted_classes_digit1 = torch.max(digit1, 1)
        _, predicted_classes_digit2 = torch.max(digit2, 1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1
            if data_classes[b + k, 0] != predicted_classes_digit1[k]:
                nb_errors_digit1 += 1
            if data_classes[b + k, 1] != predicted_classes_digit2[k]:
                nb_errors_digit2 += 1
            if ((predicted_classes_digit1[k] < predicted_classes_digit2[k]) and predicted_classes[k] == 0):
                nb_errors_comparison += 1
            if ((predicted_classes_digit1[k] > predicted_classes_digit2[k]) and predicted_classes[k] == 1):
                nb_errors_comparison += 1
            
    return nb_data_errors, nb_errors_comparison, nb_errors_digit1, nb_errors_digit2

In [ ]:
def result_compute_manually(model, data_input, data_target, data_classes, mini_batch_size):
    
    nb_data_errors = 0
    nb_errors_comparison = 0
    nb_errors_digit1 = 0
    nb_errors_digit2 = 0
    
    for b in range(0, data_input.size(0), mini_batch_size):
        digit1, digit2, result = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(result, 1)
        _, predicted_classes_digit1 = torch.max(digit1, 1)
        _, predicted_classes_digit2 = torch.max(digit2, 1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1
            if data_classes[b + k, 0] != predicted_classes_digit1[k]:
                nb_errors_digit1 += 1
            if data_classes[b + k, 1] != predicted_classes_digit2[k]:
                nb_errors_digit2 += 1
            if ((predicted_classes_digit1[k] < predicted_classes_digit2[k]) and predicted_classes[k] == 0):
                nb_errors_comparison += 1
            if ((predicted_classes_digit1[k] > predicted_classes_digit2[k]) and predicted_classes[k] == 1):
                nb_errors_comparison += 1
            
    return nb_data_errors, nb_errors_comparison, nb_errors_digit1, nb_errors_digit2

In [29]:
######################################################################
def train_model(model, train_input, train_target, train_classes, nb_epochs, mini_batch_size):
    criterion = nn.CrossEntropyLoss()
    eta = 1e-3
    optimizer = optim.SGD(model.parameters(), lr = eta)

    for e in range(nb_epochs):
        acc_loss_result = 0
        acc_loss_digit1 = 0
        acc_loss_digit2 = 0
        
        for b in range(0, train_input.size(0), mini_batch_size):
            digit1, digit2, result = model(train_input.narrow(0, b, mini_batch_size))
            loss_result = criterion(result, train_target.narrow(0, b, mini_batch_size))
            loss_digit1 = criterion(digit1, train_classes[:,0].narrow(0, b, mini_batch_size))
            loss_digit2 = criterion(digit2, train_classes[:,1].narrow(0, b, mini_batch_size))
            acc_loss_result = acc_loss + loss.item()
            acc_loss_digit1 = loss_digit1 + loss.item()
            acc_loss_digit2 = loss_digit2 + loss.item()
            model.zero_grad()
            loss.backward()
            optimizer.step()
    
        # print(e, acc_loss)

In [30]:
######################################################################            
# def eval_Model(model, mini_batch_size, nb_epochs):


In [43]:
######################################################################   
class ConvNoWS(nn.Module):
    def __init__(self):
        super(ConvNoWS, self).__init__()
        #Input channels = 1, output channels = 32
        self.layer1_first_digit = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        
        #Input channels = 1, output channels = 32
        self.layer1_second_digit = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        
        #Input channels = 32, output channels = 64
        self.layer2_first_digit = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        
        #Input channels = 32, output channels = 64
        self.layer2_second_digit = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        
        # Formula to get out_put size (in_size - kernel_size + 2*(padding)) / stride) + 1
        # first layer (14-5+2*2)/1 +1 = 14/2 = 7
        # second layer (7 -4 +2*2)/1 +1 = 8/2 = 4
        # 4 * 4 * 64 input features, 1000 output features
        self.fc1_first_digit = nn.Linear(4 * 4 * 64, 1000)
        self.fc1_second_digit = nn.Linear(4 * 4 * 64, 1000)
        
        # 1000 input features, 2 output features
        self.fc2_first_digit = nn.Linear(1000, 10)
        self.fc2_second_digit = nn.Linear(1000, 10)
        
        #Comparison of the two digits
        self.layer_comp = nn.Sequential(
            nn.Linear(20, 60),
            nn.ReLU(),
            nn.Linear(60, 120),
            nn.ReLU(),
            nn.Linear(120, 2))
        
    def forward(self, x):
        
        first_digit = x[:,[0]]
        second_digit = x[:,[1]]

        first_digit = self.layer1_first_digit(first_digit)
        second_digit = self.layer1_second_digit(second_digit)
        
        first_digit = self.layer2_first_digit(first_digit)
        second_digit = self.layer2_second_digit(second_digit)
    
        first_digit = F.relu(self.fc1_first_digit(first_digit.view(-1, 4 * 4 * 64)))
        second_digit = F.relu(self.fc1_second_digit(second_digit.view(-1, 4 * 4 * 64)))
        
        first_digit = self.fc2_first_digit(first_digit)
        second_digit = self.fc2_second_digit(second_digit)
        
        result = torch.cat((first_digit, second_digit), dim=1, out=None)
        result = self.layer_comp(result)
        
        return first_digit, second_digit, result

In [44]:
    
######################################################################   
    
train_input, train_target, train_classes, test_input, test_target, test_classes \
    = prologue.generate_pair_sets(1000)
    

# train_input, train_target, train_classes \
#     = Variable(train_input), Variable(train_target), Variable(train_classes)
# test_input, test_target, test_classes \
#     = Variable(test_input), Variable(test_target), Variable(test_classes)

def get_tests(n):
    M = []
    for k in range (0, n):
        L = []
        _, _, _, test_input, test_target, test_classes =  prologue.generate_pair_sets(1000)
        L.append(test_input)
        L.append(test_target)
        L.append(test_classes)
        M.append(L)
    return M


model = ConvNoWS()
nb_epochs = 25
mini_batch_size = 100

train_model(model, train_input, train_target, nb_epochs, mini_batch_size)
L = get_tests(10)
nb_moy_test_error = 0
nb_moy_test_comparison_errors = 0
nb_train_errors = compute_nb_errors(model, train_input, train_target, mini_batch_size)
_, nb_comparison_errors, _, _ = \
        compute_nb_errors_comparison(model, train_input, train_target, train_classes, mini_batch_size)
print('train error ConvNoWS {:0.2f}%{:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                        nb_train_errors, train_input.size(0)))
print('train comparison_error ConvNoWS {:0.2f}%{:d}/{:d}'.format((100 * nb_comparison_errors) / train_input.size(0),
                                        nb_comparison_errors, train_input.size(0)))

for k in range (0, len(L)):
    nb_test_errors = compute_nb_errors(model, L[k][0], L[k][1], mini_batch_size)
    _, nb_test_comparison_errors, _, _ = \
            compute_nb_errors_comparison(model, L[k][0], L[k][1], L[k][2], mini_batch_size)
    nb_moy_test_error += nb_test_errors
    nb_moy_test_comparison_errors += nb_test_comparison_errors
    print('test error ConvNoWS {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / L[k][0].size(0),
                                                nb_test_errors, L[k][0].size(0)))
    print('test comparison_error ConvNoWS {:0.2f}% {:d}/{:d}'.format((100 * nb_test_comparison_errors) / L[k][0].size(0),
                                                nb_test_comparison_errors, L[k][0].size(0)))
print('Average test error ConvNoWS {:0.2f}% {:0.1f}/{:d}'.format((100*nb_moy_test_error/10) / L[0][0].size(0),nb_moy_test_error/10, L[0][0].size(0) ))
print('Average test comparison_error ConvNoWS {:0.2f}% {:0.1f}/{:d}'.format((100*nb_moy_test_comparison_errors/10) / L[0][0].size(0),nb_moy_test_comparison_errors/10, L[0][0].size(0) ))

TypeError: __init__() takes from 1 to 2 positional arguments but 3 were given